## Preprocessing


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME', 'AFFILIATION'.
application_df = application_df.drop(columns=['EIN', 'NAME', 'AFFILIATION'])

application_df.head()


,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df.APPLICATION_TYPE.value_counts()

app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts<500].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df.CLASSIFICATION.value_counts()

class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1x

class_count[class_count>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count<1800].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Create bins
bin_edges = [0, 10000, 100000, 1000000, 10000000, float('inf')]
bin_labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']

# Merge bins into df
application_df['ASK_AMT_BINS'] = pd.cut(application_df['ASK_AMT'], bins=bin_edges, labels=bin_labels)
application_df.drop(columns=['ASK_AMT'], inplace=True)
application_df.head()

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_BINS
0,T10,C1000,ProductDev,Association,1,0,N,1,Very Low
1,T3,C2000,Preservation,Co-operative,1,1-9999,N,1,Moderate
2,T5,C3000,ProductDev,Association,1,0,N,0,Very Low
3,T3,C2000,Preservation,Trust,1,10000-24999,N,1,Very Low
4,T3,C1000,Heathcare,Trust,1,100000-499999,N,1,Moderate


In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT_BINS'])

app_dummies

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_BINS_Very Low,ASK_AMT_BINS_Low,ASK_AMT_BINS_Moderate,ASK_AMT_BINS_High,ASK_AMT_BINS_Very High
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
34295,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
34296,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
34297,1,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = app_dummies.drop(columns=['IS_SUCCESSFUL'], axis=1)
y = app_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model 1


In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_1 = tf.keras.models.Sequential()

# First hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=41))

# Second hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Output layer
nn_model_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3360      
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                                 
 dense_5 (Dense)             (None, 1)                 81        
                                                                 
Total params: 9921 (38.75 KB)
Trainable params: 9921 (38.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Compile the model
nn_model_1.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

# Train the model
fit_model_1 = nn_model_1.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.6223 - accuracy: 0.6455
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6137 - accuracy: 0.6506
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6100 - accuracy: 0.6529
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6091 - accuracy: 0.6536
Epoch 5/100
804/804 [==============================] - 7s 9ms/step - loss: 0.6079 - accuracy: 0.6551
Epoch 6/100
804/804 [==============================] - 4s 4ms/step - loss: 0.6069 - accuracy: 0.6557
Epoch 7/100
804/804 [==============================] - 5s 6ms/step - loss: 0.6059 - accuracy: 0.6557
Epoch 8/100
804/804 [==============================] - 5s 6ms/step - loss: 0.6051 - accuracy: 0.6557
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6053 - accuracy: 0.6582
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6045 - accura

In [18]:
# Evaluate the model using the test data
model_loss_1, model_accuracy_1 = nn_model_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_1}, Accuracy: {model_accuracy_1}")

268/268 - 1s - loss: 0.6218 - accuracy: 0.6596 - 653ms/epoch - 2ms/step
Loss: 0.6217626333236694, Accuracy: 0.6595918536186218


In [19]:
# Export our model to HDF5 file
nn_model_1.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Compile, Train and Evaluate the Model 2

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_2 = tf.keras.models.Sequential()

# First hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=40, activation="relu", input_dim=41))

# Second hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=90, activation="relu"))

# Third hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=120, activation="relu"))

# Fourth hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=150, activation="relu"))

# Fifth hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=180, activation="relu"))

# Output layer
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_2.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 40)                1680      
                                                                 
 dense_26 (Dense)            (None, 90)                3690      
                                                                 
 dense_27 (Dense)            (None, 120)               10920     
                                                                 
 dense_28 (Dense)            (None, 150)               18150     
                                                                 
 dense_29 (Dense)            (None, 180)               27180     
                                                                 
 dense_30 (Dense)            (None, 1)                 181       
                                                                 
Total params: 61801 (241.41 KB)
Trainable params: 6180

In [37]:
# Compile the model
nn_model_2.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

# Train the model
fit_model_2 = nn_model_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 5s 5ms/step - loss: 0.6246 - accuracy: 0.6449
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6141 - accuracy: 0.6528
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6108 - accuracy: 0.6557
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6089 - accuracy: 0.6543
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6080 - accuracy: 0.6581
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6068 - accuracy: 0.6580
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6056 - accuracy: 0.6584
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6049 - accuracy: 0.6575
Epoch 9/100
804/804 [==============================] - 6s 8ms/step - loss: 0.6039 - accuracy: 0.6607
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6032 - accura

In [38]:
# Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn_model_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

268/268 - 1s - loss: 0.6487 - accuracy: 0.6584 - 556ms/epoch - 2ms/step
Loss: 0.6486969590187073, Accuracy: 0.6584256291389465


In [39]:
# Export our model to HDF5 file
nn_model_2.save("AlphabetSoupCharity_Optimization.h5")

## Compile, Train and Evaluate the Model 3

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_3 = tf.keras.models.Sequential()

# First hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=41))

# Second hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=100, activation="relu"))

# Third hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=100, activation="relu"))


# Output layer
nn_model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_3.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 100)               4200      
                                                                 
 dense_22 (Dense)            (None, 100)               10100     
                                                                 
 dense_23 (Dense)            (None, 100)               10100     
                                                                 
 dense_24 (Dense)            (None, 1)                 101       
                                                                 
Total params: 24501 (95.71 KB)
Trainable params: 24501 (95.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
# Compile the model
nn_model_3.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

# Train the model
fit_model_3 = nn_model_3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6210 - accuracy: 0.6465
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6122 - accuracy: 0.6543
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6098 - accuracy: 0.6551
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6085 - accuracy: 0.6574
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6077 - accuracy: 0.6566
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6068 - accuracy: 0.6581
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6057 - accuracy: 0.6564
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6051 - accuracy: 0.6598
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6043 - accuracy: 0.6586
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6039 - accura

In [34]:
# Evaluate the model using the test data
model_loss_3, model_accuracy_3 = nn_model_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

268/268 - 1s - loss: 0.6620 - accuracy: 0.6595 - 558ms/epoch - 2ms/step
Loss: 0.6620382070541382, Accuracy: 0.6594752073287964


In [35]:
# Export our model to HDF5 file
nn_model_3.save("AlphabetSoupCharity_Optimization.h5")

## Machine Learning Analysis and Resport

Overview: The purpose of this analysis was to try to optimize the nueral network by adjustng the amount of columns, hidden layers, and neurons.

Results:
  Data Processing:
    Target for the model - "IS_SUCCESSFUL"
    Features for the model:
      "APPLICATION_TYPE",
      "CLASSIFICATION",
      "USE_CASE",
      "ORGANIZATION",
      "STATUS",
      "INCOME_AMT",
      "SPECIAL_CONSIDERATIONS",
      "ASK_AMT"
    Removed:
      "NAME",
      "EIN",
      "AFFILIATION"



Compiling, Traing, and Evaluation

Initial Model:

3 Total Layers
1st Layer - 80 neurons - RELU
2nd Layer - 30 neurons - RELU
3rd Layer - 1 neuron - SIGMOID
Loss: 0.558
Accuracy: 0.726


Optimization Model 1:
3 Total Layers
1st Layer - 80 neurons - RELU
2nd Layer - 80 neurons - RELU
3rd Layer - 1 neuron - SIGMOID
Loss: 0.622
Accuracy: 0.660


Optimization Model 2:
6 Total Layers
1st Layer - 40 neurons - RELU
2nd Layer - 90 neurons - RELU
3rd Layer - 120 neurons - RELU
4rd Layer - 150 neurons - RELU
5rd Layer - 180 neurons - RELU
6th Layer - 1 neuron - SIGMOID
Loss: 0.650
Accuracy: 0.658


Optimization Model 3:
4 Total Layers
1st Layer - 100 neurons - RELU
2nd Layer - 100 neurons - RELU
3rd Layer - 100 neurons - RELU
4th Layer - 1 neurons - SIGMOID

Loss: 0.662
Accuracy: 0.659


Summary
The best this model could achieve was 72.6% Accuracy rating. Through my optimizations the model actuall peformed with worse with accuracy scores all being less than 70% even though more nuerons were used. This leads me to believe that the feature I removed in my efforts to optimize the data was more important in regards to the outcome than I initially thought. In the future a different column should probably be removed to improve accuracy scores.